In [6]:
#Import Required Libraries:
import pandas as pd
import networkx as nx
import json

#Load Author Network Data
network_df = pd.read_csv(r"C:\DV_major3\data_scopus.csv")

nodes = network_df['EID'].values
network_df=network_df.fillna(0)
network_df.head(5)

,Title,Year,EID,Abstract,Publisher,Conference name,Conference date,Authors,Author(s) ID,Authors with affiliations,Source title,Abbreviated Source Title,Cited by
0,Virtual reality applications for the built env...,2020,2-s2.0-85086464158,With its advanced capabilities of immersive an...,Elsevier B.V.,0,0,"Zhang Y., Liu H., Kang S.-C., Al-Hussein M.",57202536919;53984550800;57215426514;6603541102;,"Zhang, Y., Department of Civil and Environment...",Automation in Construction,Autom Constr,0.0
1,"Self-tracking while doing sport: Comfort, moti...",2020,2-s2.0-85082875828,The spread of wearable technologies is paving ...,Academic Press,0,0,"Rapp A., Tirabeni L.",23398572100;57191836654;,"Rapp, A., Computer Science Department, Univers...",International Journal of Human Computer Studies,Int J Hum Comput Stud,0.0
2,"Bridge damage: Detection, IFC-based semantic e...",2020,2-s2.0-85078194587,Building Information Modeling (BIM) representa...,Elsevier B.V.,0,0,"Isailović D., Stojanovic V., Trapp M., Richter...",57205293410;56421700900;24831175200;3619515940...,"Isailović, D., Department for Construction Pro...",Automation in Construction,Autom Constr,0.0
3,VR system for spatio-temporal visualization of...,2019,2-s2.0-85075706132,Social media analysis is helpful to understand...,Springer,0,0,"Okada K., Yoshida M., Itoh T., Czauderna T., S...",57201502480;57171240600;35373203100;2592569550...,"Okada, K., Ochanomizu University, Tokyo, Japan...",Multimedia Tools and Applications,Multimedia Tools Appl,0.0
4,DiseaSE: A biomedical text analytics system fo...,2019,2-s2.0-85074886243,Due to increasing volume and unstructured natu...,Academic Press Inc.,0,0,"Abulaish M., Parwez M.A., Jahiruddin",6505934038;57202719622;35590173900;,"Abulaish, M., Department of Computer Science, ...",Journal of Biomedical Informatics,J. Biomed. Informatics,0.0


In [7]:
network_df = network_df.dropna(subset=['Author(s) ID','Year','Authors','Authors with affiliations'])

# nodes = df['Author(s) ID'].values
network_df = network_df.fillna(0)
G = nx.Graph()
nodes = {}


for _, row in network_df.iterrows():
    # Extract authors from the "Authors" column
    authors = row['Authors'].split(',')
    author_Id = row['Author(s) ID'].split(';')
    Title=row['Title']
    Year=row['Year']
    Citations=row['Cited by']
    Publisher=row['Publisher']
    Authorwa=row['Authors with affiliations']

    # Add authors as nodes

    for author in range(len(authors)):

        aid= author_Id[author]
        aname= ';'.join(authors)
        title=Title

        if(aid!=""):
          #print(aid)
         nodes={'id':aid,
          "Authors": aname,
          "Title": title,
          "Year": Year,
          "Citations": Citations,
          "Publisher": Publisher,
          "Author with affiliations":Authorwa
           }
         G.add_node(aid,**nodes)

print(G)
from networkx.readwrite import json_graph
with open("publicationsNetworkGraph.json", "w") as f:
    json.dump(json_graph.node_link_data(G), f)


Graph with 1553 nodes and 0 edges


In [9]:
# Load the data from the JSON file
with open('publicationsNetworkGraph.json', 'r') as infile:
    data1 = json.load(infile)
newnodes=[]
# Create a NetworkX graph
G=nx.Graph()

def get_author_country(authors_with_affiliations):
    first_affiliation = authors_with_affiliations.split(';')[0].strip()
    country = first_affiliation.split(',')[-1].strip()
    return country
for node_data in data1['nodes']:
    eid = node_data.get('id')
    authors=node_data.get('Authors')
    title = node_data.get('Title')
    year = node_data.get('Year')
    citations = node_data.get('Citations')
    publisher = node_data.get('Publisher')

    authors_with_affiliations = str(node_data.get('Author with affiliations'))

    country = get_author_country(authors_with_affiliations)

    G.add_node(eid,authors=authors, title=title, year=year, citations=citations, publisher=publisher,authors_with_affiliations=authors_with_affiliations, country=country)
# Generate clusters based on authors' countries
clusters = list(nx.connected_components(G))

# Assign classes to each cluster
class_mapping = {node: idx for idx, cluster in enumerate(clusters) for node in cluster}
nx.set_node_attributes(G, class_mapping, 'class')

# title = data["Title"]
# authors_info = data["Authors with affiliations"].split(';')

# Create a dictionary to store co-authorship relationships
coauthorship_dict = {}
for row in network_df.iterrows():
    authors = row[1]['Author(s) ID'].split(';')
    for i in range(len(authors)):
      for j in range(i+1, len(authors)):
        if(authors[i]!="" and authors[j]!=""):
          G.add_edge(authors[i], authors[j])


print(G)
# print(edges)


# Save the data as a JSON file with proper character encoding
coauthorshipdata = {'nodes': [{'id': node, **G.nodes[node]} for node in G.nodes()],'links': [{'source': source, 'target': target} for source, target in G.edges()]}
#coauthorship_data = {'nodes': [{'id': node, **G.nodes[node]} for node in G.nodes()],'edges': edges}
with open('coauthorshipdata.json', 'w', encoding='utf-8') as outfile:
    json.dump(coauthorshipdata, outfile, ensure_ascii=False)

Graph with 1553 nodes and 3049 edges
